In [1]:
%%capture
!git clone https://github.com/HackerBschor/SofteningQueryEvaluation
%cd SofteningQueryEvaluation

!pip3 install faiss-gpu-cu12
!pip3 install pgvector
!pip3 install kagglehub

In [2]:
from huggingface_hub import notebook_login
notebook_login()

# Evaluate Similarity Grouping


In [3]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from models import ModelMgr
from models.embedding.SentenceTransformer import SentenceTransformerEmbeddingModel
from models.semantic_validation import LLaMAValidationModel

from db.operators import Dummy, Select
from db.criteria import SoftEqual
from db.structure import Column, Constant
import kagglehub

from evaluation.util import calculate_metrics, calc_bleu

import time

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
# Models
# meta-llama/Llama-3.2-3B-Instruct
# meta-llama/Meta-Llama-3-8B-Instruct

stem = SentenceTransformerEmbeddingModel(ModelMgr())
lsv = LLaMAValidationModel(ModelMgr(), model_path = "meta-llama/Meta-Llama-3-8B-Instruct")
# lsv = DeepSeekValidationModel(m)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [5]:
ZERO_SHOT_SYSTEM_PROMPT = "You are a validator. Respond with \"no\" and \"yes\" only!"
ZERO_SHOT_PROMPTING_TEMPLATE = 'Does "{}" describes "{}"'

random_state = 420
max_number_categories = 10

target = "category_2"

In [6]:
path = kagglehub.dataset_download("lakritidis/product-classification-and-categorization")

df = pd.read_csv(f"{path}/pricerunner_aggregate.csv", header=None, index_col=0)
df.drop(columns=[2, 3, 5], inplace=True)
df.columns = ["product", "category_1", "category_2"]
df.head()

100%|██████████| 22.8M/22.8M [00:02<00:00, 10.1MB/s]

Extracting files...


,product,category_1,category_2
0,,,
1,apple iphone 8 plus 64gb silver,Apple iPhone 8 Plus 64GB,Mobile Phones
2,apple iphone 8 plus 64 gb spacegrau,Apple iPhone 8 Plus 64GB,Mobile Phones
3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...,Apple iPhone 8 Plus 64GB,Mobile Phones
4,apple iphone 8 plus 64gb space grey,Apple iPhone 8 Plus 64GB,Mobile Phones
5,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,Apple iPhone 8 Plus 64GB,Mobile Phones


In [7]:
categories = pd.Series(df[target].unique())
categories = categories.sample(min(len(categories), max_number_categories)).tolist()

print(len(categories)), print(categories)

10
['Dishwashers', 'CPUs', 'Fridge Freezers', 'Digital Cameras', 'Microwaves', 'Washing Machines', 'Mobile Phones', 'Fridges', 'TVs', 'Freezers']


(None, None)

In [8]:
df_reduced = pd.concat([df[df[target] == category].sample(max_number_categories, random_state=random_state) for category in categories])

gt = {(x["product"], x[target]) for key, x in df_reduced.iterrows()}
print(str(gt)[0:100], "...", len(gt))

{('bosch gsn58aw30 freistehend senkrecht 360l a', 'Freezers'), ('sony xperia xa2 ultra black', 'Mobi ... 100


In [9]:
overall_result = {}

def evaluate(method, threshold, system_prompt=ZERO_SHOT_SYSTEM_PROMPT, prompt_template=ZERO_SHOT_PROMPTING_TEMPLATE):
    pred = []
    runtimes = []
    for category in tqdm(categories):
        d = Dummy("products", ["product"], [(x[0], ) for x in gt])
        s = Select(d, SoftEqual(Column("product"),Constant(category), method=method, em=stem, sv=lsv, threshold=threshold, zfs_system_prompt=system_prompt, zfs_prompt_template = prompt_template))

        tic = time.time()
        result = s.open().fetch_all()
        toc = time.time()

        pred.extend([(x["product"], category) for x in result])
        runtimes.append(toc - tic)

    scores = calculate_metrics(gt, set(pred), np.mean(runtimes))

    print(method, threshold, scores["F1 Score"])

    return scores, pred

In [10]:
for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    res, _ = evaluate("threshold", thresh)
    overall_result[("threshold", thresh)] = res
    if res["Recall"] == 0.0:
        break

100%|██████████| 10/10 [00:13<00:00,  1.39s/it]


threshold 0.1 0.2680115273775216


100%|██████████| 10/10 [00:12<00:00,  1.28s/it]


threshold 0.2 0.5478547854785478


100%|██████████| 10/10 [00:12<00:00,  1.29s/it]


...

threshold 0.4 0.6067415730337079


100%|██████████| 10/10 [00:12<00:00,  1.28s/it]


threshold 0.5 0.21666666666666667


100%|██████████| 10/10 [00:12<00:00,  1.28s/it]

threshold 0.6 0


In [11]:
res, pred = evaluate("zero-few-shot", None)
overall_result[("zero-few-shot", None)] = res

100%|██████████| 10/10 [01:28<00:00,  8.82s/it]

zero-few-shot None 0.7351778656126482


In [12]:
for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    res, _ = evaluate("both", thresh)
    overall_result[("both", thresh)] = res
    if res["Recall"] == 0.0:
        break

100%|██████████| 10/10 [01:02<00:00,  6.25s/it]


both 0.1 0.7426160337552743


100%|██████████| 10/10 [00:29<00:00,  2.99s/it]


both 0.2 0.7570093457943925


100%|██████████| 10/10 [00:23<00:00,  2.30s/it]


...

both 0.8 0


100%|██████████| 10/10 [00:12<00:00,  1.28s/it]


both 0.9 0


100%|██████████| 10/10 [00:13<00:00,  1.32s/it]

both 1.0 0


In [13]:
for key in tqdm(overall_result):
    scores_bleu = calc_bleu(gt, overall_result[key]["pred"])
    for score_bleu in scores_bleu:
        overall_result[key][score_bleu] = scores_bleu[score_bleu]

100%|██████████| 17/17 [01:02<00:00,  3.69s/it]


In [14]:
df_results = pd.DataFrame([{"method": k[0], "threshold": k[1]} | v for k,v in overall_result.items()])
df_results

,method,threshold,Precision,Recall,F1 Score,tp,fn,fp,runtime,pred,bleu1,bleu2,bleu3,bleu4
0,threshold,0.1,0.156566,0.93,0.268012,93,7,501,1.389067,"{(beko wtg620m1w 6kg washing machine, TVs), (p...",0.987544,0.986159,0.984264,0.982120
1,threshold,0.2,0.408867,0.83,0.547855,83,17,120,1.277390,{(philips 55hfl7011t/12 55 inch signature prof...,0.908404,0.892725,0.877921,0.862979
2,threshold,0.3,0.628099,0.76,0.687783,76,24,45,1.285728,"{(cello 22 c22230t2 led tv c22230t2 mw01, TVs)...",0.839222,0.819443,0.800086,0.784812
3,threshold,0.4,0.692308,0.54,0.606742,54,46,24,1.284673,"{(cello 22 c22230t2 led tv c22230t2 mw01, TVs)...",0.701566,0.671405,0.638418,0.611531
4,threshold,0.5,0.650000,0.13,0.216667,13,87,7,1.278600,{(indesit ld70 n1 s fridge freezer stainless s...,0.378740,0.306038,0.265724,0.242154
5,threshold,0.6,0.000000,0.00,0.000000,0,100,1,1.283839,{(fridgemaster mtz55183ff tall frost free free...,0.123497,0.061672,0.045768,0.035064
6,zero-few-shot,NaN,0.607843,0.93,0.735178,93,7,60,8.822343,{(bosch gsn58aw30 freistehend senkrecht 360l a...,0.967009,0.962819,0.956123,0.949348
7,both,0.1,0.642336,0.88,0.742616,88,12,49,6.248804,{(bosch gsn58aw30 freistehend senkrecht 360l a...,0.938321,0.929470,0.919465,0.909976
8,both,0.2,0.710526,0.81,0.757009,81,19,33,2.986085,"{(sony xperia xa2 ultra black, Mobile Phones),...",0.878237,0.859946,0.842206,0.829733
9,both,0.3,0.728155,0.75,0.738916,75,25,28,2.300546,"{(cello 22 c22230t2 led tv c22230t2 mw01, TVs)...",0.826927,0.806673,0.785388,0.771891


In [15]:
df_results.to_pickle("Products_mpnetBaseV2_LLama8B.pkl")
df_results.drop(columns=["pred"]).to_csv("Products_mpnetBaseV2_LLama8B.csv")